## Transactions
#### 03.4 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2024-02-14 (started 2021-12-19)

- Load credentials
- Connect to the blockchain
- Execute a first payment transaction using Python
- Add a note to the payment

## Setup
Starting with this chapter 3.4, the lines below will always automatically load ...
* The accounts MyAlgo, Alice, Bob, Charlie, Dina
* The API credentials
* The functions in `algo_util.py`
    * These functions can be found in the folder `sharedCode` which is two levels up

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
import algosdk.error
import json

In [3]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI
ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU
BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU
CHARLS5INIT5KOLDK3F2QAZXY7N4W7GOE6UTXHEHS3RBNCR2WJXRRTH3GY
DINAAZWBJM6DRY73CWK7IOA4S7C6PVXQMQ4DJ3TK3DAGAGFJTX5KEYESDI


## Connecting to the Algorand Blockchain via API
+ We choose to connect via Algonode API
- API **address** stored ...
    - For the testnet in `cred['algod_test']`
    - For the mainnet in `cred['algod_main']`
- API **crendentials** stored in `cred['api_token']` (currently not needed)

In [4]:
# Today we work with the testnet
algod_token   = ''                        # Only needed if we have our own server
algod_address = cred['algod_test']        # TESTNET, alternatively cred['algod_main'] 
api_token = cred['api_token']             # Currently empty, this may change

# Initialize the algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address, headers=api_token)

#### Test the connection
- Our first Python access of the blockchain
- What's the last block?
    - Note that block count on testnet is larger (Why?)
    - Check here (select *testnet* on top left): https://app.dappflow.org/explorer/home

In [5]:
algod_client.status()["last-round"]

37363418

### Check the accounts on the blockchain
#### (a) on the Mainnet
- Mainnet https://explorer.perawallet.app
- Check the link below to find the Algos that you have received at the beginning of class

In [6]:
# Create a link to directly access your MyAlgo account
print(cred['explore_main']+'address/'+MyAlgo['public'])

https://explorer.perawallet.app/address/WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI


#### (b) on the Testnet
* Accounts, program code etc work just like on the main net
* Notice that your `MyAlgo` account on the testnet has still 0 Algos

In [7]:
print(cred['explore_test']+'address/'+MyAlgo['public'])

https://testnet.explorer.perawallet.app/address/WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI


### Fund with testnet Algos

Try any of these:
- https://bank.testnet.algorand.network/
- https://dispenser.testnet.aws.algodev.network
- Fund `MyAlgo`, `Alice` and `Bob`. How many test ALGOs did you get?

### Obtain holdings
* The `algod_client.account_info()` function obtains all sorts of information about an account
    * For more details, see chapter 11 (empirics)

In [8]:
# Get holdings of testnet Algos
address = Alice["public"]
algod_client.account_info(address)["amount"]

10712999

In [9]:
# Holdings are in micro Algo ... convert
algo_precision = 1e6
algo_amount    = algod_client.account_info(address)["amount"]/algo_precision
print(f"Address {address} has {algo_amount} test algos")

Address ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU has 10.712999 test algos


## A first payment transaction

#### The suggested parameters for a transaction (on the test network)
* Get standard parameters from API
* Simplifies creation of a transaction
* Part of ever y**Step 1 – prepare transaction**
    * `first` and `last` are suggested params for first and last round valid. Compare to current round
    * Suggested fee is standard fee
    * `gh` stands for genesis hash (=first entry on the blockchain)

In [10]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

{
    "first": 37363420,
    "last": 37364420,
    "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "gen": "testnet-v1.0",
    "fee": 0,
    "flat_fee": false,
    "consensus_version": "https://github.com/algorandfoundation/specs/tree/925a46433742afb0b51bb939354bd907fa88bf95",
    "min_fee": 1000
}


#### Step 1: prepare and create unsigned transaction using `PaymentTxn`

In [11]:
# Parameters
sp        = algod_client.suggested_params()       # suggested params
amount    = 0.1                                   # in ALGO
algo_prec = 1e6
amt_microalgo = int(amount * algo_prec)     # in Micro-ALGO 

# Create (unsigned) TX
txn = PaymentTxn(sender = Alice['public'],     # <--- From
                 sp = sp,                      # <---- ALWAYS include the sp
                 receiver = Bob['public'],     # <---- To
                 amt = amt_microalgo           # <---- in Micro-ALGO
                )

In [12]:
# Interesting: this is how a transaction looks like
print(json.dumps(vars(txn), indent=4))

{
    "sender": "ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU",
    "fee": 1000,
    "first_valid_round": 37363421,
    "last_valid_round": 37364421,
    "note": null,
    "genesis_id": "testnet-v1.0",
    "genesis_hash": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "group": null,
    "lease": null,
    "type": "pay",
    "rekey_to": null,
    "receiver": "BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU",
    "amt": 100000,
    "close_remainder_to": null
}


In [13]:
# Interesting: we already have a txid
print(txn.get_txid())

JWYC5JGX2SLNKJOVEE62MNATEZX7VTD4AXVI2YYXGTLMV5TD7N4A


In [14]:
# Is it already on the blockchain? 
# No ... we have not yet sent sent it --> 404 error
print(cred['explore_test']+'tx/'+txn.get_txid())

https://testnet.explorer.perawallet.app/tx/JWYC5JGX2SLNKJOVEE62MNATEZX7VTD4AXVI2YYXGTLMV5TD7N4A


#### Step 2: sign

In [15]:
stxn = txn.sign(Alice['private'])                 # <---- Alice signs with private key

In [16]:
# Interesting: this is how a transaction looks like
# The new stxn object consists of
print(stxn.transaction)                           # same as above
print('')
print(stxn.signature)                             # signature signs the transaction

{'sender': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'fee': 1000, 'first_valid_round': 37363421, 'last_valid_round': 37364421, 'note': None, 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'amt': 100000, 'close_remainder_to': None}

d5NCHq/UD06INZO76umYOH+xbpeA50YeF5c0dxkXHNZ6gUUgr6WCYyJegj/ktsm3PXuOEKSoWtmEIOxUOixnBw==


In [17]:
# Interesting: The transaction ID is the same as before
# ... and still nothing on the blockchain
print(stxn.get_txid())
print(cred['explore_test']+'tx/'+stxn.get_txid())

JWYC5JGX2SLNKJOVEE62MNATEZX7VTD4AXVI2YYXGTLMV5TD7N4A
https://testnet.explorer.perawallet.app/tx/JWYC5JGX2SLNKJOVEE62MNATEZX7VTD4AXVI2YYXGTLMV5TD7N4A


#### Step 3: send

In [18]:
txid = algod_client.send_transaction(stxn)

In [19]:
# Transaction just asent to the blockchain
# Does not yet contain a `confirmed-round` object
txinfo = algod_client.pending_transaction_info(txid)
print(txinfo)

{'pool-error': '', 'txn': {'sig': 'd5NCHq/UD06INZO76umYOH+xbpeA50YeF5c0dxkXHNZ6gUUgr6WCYyJegj/ktsm3PXuOEKSoWtmEIOxUOixnBw==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 37363421, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 37364421, 'rcv': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'snd': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'type': 'pay'}}}


#### Step 4: Wait for confirmation

In [20]:
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  37363425.
Waiting for round 37363425 to finish.
Transaction JWYC5JGX2SLNKJOVEE62MNATEZX7VTD4AXVI2YYXGTLMV5TD7N4A confirmed in round 37363426.


In [21]:
# Note that txinfo has now a 'confirmed-round'
print(txinfo)

{'confirmed-round': 37363426, 'pool-error': '', 'txn': {'sig': 'd5NCHq/UD06INZO76umYOH+xbpeA50YeF5c0dxkXHNZ6gUUgr6WCYyJegj/ktsm3PXuOEKSoWtmEIOxUOixnBw==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 37363421, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 37364421, 'rcv': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'snd': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'type': 'pay'}}}


In [22]:
# Now we can check the tx also on algoexplorer
print(cred['explore_test']+'tx/'+txid)

https://testnet.explorer.perawallet.app/tx/JWYC5JGX2SLNKJOVEE62MNATEZX7VTD4AXVI2YYXGTLMV5TD7N4A


## Add a note to a transaction
* Create a transaction a bit more efficiently
* Add a transaction note

In [23]:
# Step 1a: Prepare
sp     = algod_client.suggested_params()       # suggested params
amount = 0.1
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Step 1b: The note (need to encode as bytes)
note_txt  = "Paying back for last dinner"
note_byte = note_txt.encode()

In [24]:
# Step 1c: create (unsigned) TX
txn = PaymentTxn(sender=Alice['public'],
                 sp = sp,
                 receiver = Bob['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )
print(txn)

{'sender': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'fee': 1000, 'first_valid_round': 37363426, 'last_valid_round': 37364426, 'note': b'Paying back for last dinner', 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'amt': 100000, 'close_remainder_to': None}


In [25]:
# Step 2+3: sign and send TX
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
print(txid)

7CNIYQZW4KH7DGW44BJ55OK3D625TCR5BRK2ALHQSUEX6V2PZPYQ


In [26]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  37363427.
Waiting for round 37363427 to finish.
Transaction 7CNIYQZW4KH7DGW44BJ55OK3D625TCR5BRK2ALHQSUEX6V2PZPYQ confirmed in round 37363428.


In [27]:
print(txinfo)

{'confirmed-round': 37363428, 'pool-error': '', 'txn': {'sig': 'BZ3kmjz9g7NM7hYgNqwRDTle27pq7vSBVDogKZdkLLBcRgQoK0sZ2pxqhEi/PFYBGu1VTenpy4rSrzexKxwhDg==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 37363426, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 37364426, 'note': 'UGF5aW5nIGJhY2sgZm9yIGxhc3QgZGlubmVy', 'rcv': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'snd': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'type': 'pay'}}}


In [28]:
# Check on Algoexplorer
# The note is readable in plain text
print(cred['explore_test']+'tx/'+txid)

https://testnet.explorer.perawallet.app/tx/7CNIYQZW4KH7DGW44BJ55OK3D625TCR5BRK2ALHQSUEX6V2PZPYQ


### Step 5 (check): Extract message in txinfo and convert back to plain text

In [29]:
import base64
note_base64 = txinfo['txn']['txn']['note']
print(note_base64)
note_byte   = base64.b64decode(note_base64)
print(note_byte)
note_txt   = note_byte.decode()
print(note_txt)

UGF5aW5nIGJhY2sgZm9yIGxhc3QgZGlubmVy
b'Paying back for last dinner'
Paying back for last dinner


## Exercises

**Exercise 1:** Send 0.82 ALGO from Dina to Alice with a thank you note

In [30]:
# Your Python code goes here

# Step 1: Prepare
sp        = algod_client.suggested_params()       # suggested params
amount    = 0.82
algo_prec = 1e6
amt_microalgo = int(amount * algo_prec)
note_txt  = "Thank you"
note_byte = note_txt.encode()

txn = PaymentTxn(sender=Dina['public'],
                 sp=sp, 
                 receiver = Alice['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )

# Step 2+3: sign and send TX
stxn = txn.sign(Dina['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  37363429.
Waiting for round 37363429 to finish.
Waiting for round 37363430 to finish.
Transaction JW2IBCLCHZSN4WVA7LSHV6GIU2TIPNEJEY2CGOYAX3N4NHBEDSNA confirmed in round 37363431.


**Exercise 2:** Obtain the (new) ALGO holdings of Alice using Python

In [31]:
# Your Python code goes here
address = Alice["public"]
algod_client.account_info(address)["amount"] / 1e6

11.330999

## Things that do not and will not work
* Following are a few things that deliberately don't work.
* Goal: learn how error messages look like and how to deal with them.

In [32]:
# Need to import this to be able to read error messages
import sys, algosdk.error

### Overspending
* Alice sends more than she owns
* The error message is very long. Scroll down to the end.

In [33]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [34]:
# Step 3b: Send
txid = algod_client.send_transaction(signed_txn)

AlgodHTTPError: TransactionPool.Remember: transaction SGQFLBDKPPN4GZGZVCEGB3FVAYF7RDZE25RXYF5KA6A43AGGKT5Q: overspend (account ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU, data {AccountBaseData:{Status:Offline MicroAlgos:{Raw:11329999} RewardsBase:27521 RewardedMicroAlgos:{Raw:0} AuthAddr:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ TotalAppSchema:{_struct:{} NumUint:26 NumByteSlice:9} TotalExtraAppPages:0 TotalAppParams:5 TotalAppLocalStates:5 TotalAssetParams:1 TotalAssets:3 TotalBoxes:0 TotalBoxBytes:0} VotingData:{VoteID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] SelectionID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] StateProofID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] VoteFirstValid:0 VoteLastValid:0 VoteKeyDilution:0}}, tried to spend {100000000})

#### Can we *catch the error* and get a better structured error message?
* **Note:** error occurs when sending the transaction

In [35]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [36]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                   # print entire error message
    if ("overspend" in str(err)):                # check for specific type of error
        print("Overspend error")         
    txid = None

TransactionPool.Remember: transaction G5SYWYTQT2HEC2D35KWB53PT7Z55JV36GG26FQGUJKWCHOZVE7NQ: overspend (account ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU, data {AccountBaseData:{Status:Offline MicroAlgos:{Raw:11329999} RewardsBase:27521 RewardedMicroAlgos:{Raw:0} AuthAddr:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ TotalAppSchema:{_struct:{} NumUint:26 NumByteSlice:9} TotalExtraAppPages:0 TotalAppParams:5 TotalAppLocalStates:5 TotalAssetParams:1 TotalAssets:3 TotalBoxes:0 TotalBoxBytes:0} VotingData:{VoteID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] SelectionID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] StateProofID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] VoteFirstValid:0 VoteLastValid:0 VoteKeyDilution:0}}, tried to spend {100000000})
Overspend error


#### What happens if we wait for the failed transaction to complete?

In [37]:
# Step 4: Wait for confirmation
try:
    txinfo = wait_for_confirmation(algod_client, txid)
    print(txinfo)
except TypeError as err:                                       # obtain error message
    # print entire error message (rather cryptic!)
    print(err)
    # Give better error message
    print("txid is empty")

Empty transaction id.
None


### Wrong signature
Bob tries to sign a transaction from Alice to Bob

In [38]:
# Step 1: prepare
sp        = algod_client.suggested_params()
algo_prec = 1e6
sender    = Alice['public']
receiver  = Bob['public']
amount    = 0.1
amount_microalgo = int(amount * algo_prec)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Bob['private'])                # <----------------- wrong person signs!

In [39]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("should have been authorized" in str(err)):                # check for specific type of error
        print("Wrong signature error")         
    txid = None

TransactionPool.Remember: transaction 3W2AUVO7KP6G7YWZR42ZIQZ6MVM5UZEKERZM7TUMPYA4AM6PIPGQ: should have been authorized by ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU but was actually authorized by BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU
Wrong signature error


### Sending the *indentical* transaction twice
* It is not possible to send the *identical* transaction twice
    * Reason: the transaction ID is calculated from the transaction data
* "Identical" means ...
    * same Sender
    * same Recipient
    * same Ammount
    * same (suggested) parameters (including first/last round) $\leftarrow$ change after 2-3 seconds

In [40]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

In [41]:
# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
    print("Submitted with txID: {}".format(txid))
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("transaction already in ledger" in str(err)):                # check for specific type of error
        print("Identical transaction {} has been submitted twice.".format(signed_txn.get_txid()))         
    txid = None    # check for specific type of error

Submitted with txID: 3W2AUVO7KP6G7YWZR42ZIQZ6MVM5UZEKERZM7TUMPYA4AM6PIPGQ


**REPEAT** only step 2-3 $\rightarrow$ error message<br>
**REPEAT** only step 1-3 $\rightarrow$ no error <br>

#### See how the sp change
* Re-run this after 2-3 seconds

In [42]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

{
    "first": 37363435,
    "last": 37364435,
    "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "gen": "testnet-v1.0",
    "fee": 0,
    "flat_fee": false,
    "consensus_version": "https://github.com/algorandfoundation/specs/tree/925a46433742afb0b51bb939354bd907fa88bf95",
    "min_fee": 1000
}


## Appendix: Useful functions
* The following functions are included in `algo_util.py`

In [43]:
def wait_for_confirmation(client, txid):
    # client = algosdk client
    # txid = transaction ID, for example from send_payment()
    # An ufficial Algorand function

    txinfo = client.pending_transaction_info(txid)       # obtain transaction information
    current_round = algod_client.status()["last-round"]        # obtain last round number
    print("Current round is  {}.".format(current_round))
    
    # Wait for confirmation
    while ( txinfo.get('confirmed-round') is None ):            # condition for waiting = 'confirmed-round' is empty
        print("Waiting for round {} to finish.".format(current_round))
        algod_client.status_after_block(current_round)             # this wait for the round to finish
        txinfo = algod_client.pending_transaction_info(txid)    # update transaction information
        current_round += 1

    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

### *Python dict as transaction note
* For attaching a Python dict as message to a note
* Example `note_dict = {'from' : 'Bob', 'to' : 'Alice', 'message' : 'Many thanks'}

In [44]:
def note_encode(note_dict):
    # note dict ... a Python dictionary
    note_json = json.dumps(note_dict)
    note_byte = note_json.encode()     
    return(note_byte)

In [45]:
def note_decode(note_64):
    # note64 =  note in base64 endocing
    # returns a Python dict
    import base64
    message_base64 = txinfo['txn']['txn']['note']
    message_byte   = base64.b64decode(message_base64)
    message_json   = message_byte.decode()
    message_dict   = json.loads( message_json )
    return(message_dict)